In [58]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone
'''
import pandas as pd

df_txexEmail = pd.read_csv('TexasExes.csv')
print("original count ",len(df_txexEmail))
df_txexEmail = df_txexEmail[['EID','Email','Personal Email','Alternate Email','Work Email']]
print(df_txexEmail.head(3))

df_Email = df_txexEmail.dropna(how='all', subset=['Email','Personal Email','Alternate Email','Work Email'])
print("Count records after dropping records with no emails : ",len(df_Email))

df_Email_nb = df_Email.dropna(subset=['Email'])
print("Count records with values in column email : ",len(df_Email_nb))

del df_Email_nb, df_txexEmail


C:\Users\le5752\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8,11,12,13,14,15,17,19,20,21,22,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


original count  166759
        EID                 Email        Personal Email Alternate Email  \
0    evr253    evreid47@gmail.com    evreid47@gmail.com             NaN   
1     cen95  cnowland54@gmail.com  cnowland54@gmail.com             NaN   
2  goldenr1         bak@campr.com                   NaN             NaN   

      Work Email  
0            NaN  
1            NaN  
2  bak@campr.com  
Count records after dropping records with no emails :  105992
Count records with values in column email :  105991


In [60]:
df_Email = df_Email[['EID','Email']]
df_Email['EID'] = df_Email['EID'].str.upper()

In [61]:
raw_eid = df_Email['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [62]:
import cx_Oracle
connection = cx_Oracle.connect("vipapp/ZJf3__6qjRt8@exa1-scan.austin.utexas.edu:1525/viprp_pdb.austin.utexas.edu")
cursor = connection.cursor()
df_edb = pd.DataFrame(columns = ['EID','Email'])
for i in range(105):
    cursor.execute("select UT_EID, vi_email_addr from vi_vips_main m, vi_vips_email_info e where m.ALUMNI_ID = e.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Email'])
    df_edb = df_edb.append(df1,ignore_index=True)
cursor.close()
connection.close()

In [63]:
print(df_edb.head(3))
print(len(df_edb))
df_ecsv = df_Email
print(df_ecsv.head(2))
print(len(df_ecsv))
del df_Email

        EID                 Email
0  0979CAYG     19jam61@gmail.com
1    AGK446  a.kaufman@utexas.edu
2    AGK446  a.kaufman@utexas.edu
143556
      EID                 Email
0  EVR253    evreid47@gmail.com
1   CEN95  cnowland54@gmail.com
105992


In [64]:
df_email = pd.DataFrame()
for i in range(len(df_ecsv)):
    id_csv = df_ecsv.iloc[i,0]
    email_csv = df_ecsv.iloc[i,1]
    email_db = (df_edb.loc[df_edb['EID'] == id_csv]).iloc[:,1]
    email_db = {*email_db}
    email_csv = {email_csv}
    diff = (email_csv - email_db)
    if len(diff) > 0:
        #df_email = df_email.append({'EID':id_csv, 'Email':diff}, ignore_index=True)
        df_email = df_email.append({'EID':id_csv, 'Email_csv':df_ecsv.iloc[i,1], 'Email_db':email_db}, ignore_index=True)

In [65]:
print(df_email.head(4))
print(len(df_email))
#df_email = df_email.drop_duplicates()
print(len(df_email))

       EID                    Email_csv                     Email_db
0    CEN95         cnowland54@gmail.com      {cnowland@eanesisd.net}
1    SFS99     shahatitbureau@yahoo.com         {shahatit@orange.jo}
2   CNG829  caitlinnoelgraham@gmail.com      {caitgraham@utexas.edu}
3  JMB8542         jrburmicky@gmail.com  {jorge_burmicky@utexas.edu}
10372
10372


In [55]:
df_email.to_csv(r'C:\Users\le5752\Documents\DB_Python\EmailsDiff.csv', index = False)